<a href="https://colab.research.google.com/github/ZeleOeO/Artificial-Intelligence-Projects/blob/main/Negative_Positive_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

In [2]:
import pandas as pd
import numpy as np

import tensorflow
from tensorflow import keras

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

Initialize the tokenizer and read the data from the csv

In [3]:
tokenizer = Tokenizer(oov_token="<OOV_Token>")


dataset = pd.read_csv("/content/data.csv")

sentences = dataset["Sentence"]
labels = dataset["Sentiment"]

print(type(sentences))
print(dataset.shape)

<class 'pandas.core.series.Series'>
(5842, 2)


In [4]:
training_size = 4000
num_epochs = 30
vocab_size = 5000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'

In [5]:
training_sentences = sentences[0:training_size]
training_labels = labels[0:training_size]

testing_sentences = sentences[training_size:]
testing_labels = labels[training_size:]

In [6]:
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, padding=padding_type, truncating = trunc_type, maxlen=max_length)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, padding=padding_type, truncating = trunc_type, maxlen=max_length)

Convert the labels and the padded texts to a numpy array

In [7]:
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)

Create the model with an embedder and then fit it

In [12]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(24, activation = "relu"),
    keras.layers.Dense(1, activation = "softmax")
])

model.compile(loss="categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 16)           80000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 24)                408       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 80,433
Trainable params: 80,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)